### Задание 1.
Обучите нейронную сеть решать шифр Цезаря.

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random

# Шаг 1: Создаем алгоритм шифра Цезаря с заданным сдвигом K (в данном случае K = 3)
def caesar_cipher(text, shift):
    alphabet = string.ascii_lowercase
    encrypted = ""
    for char in text:
        if char in alphabet:
            index = alphabet.find(char)
            new_index = (index + shift) % len(alphabet)
            encrypted += alphabet[new_index]
        else:
            encrypted += char
    return encrypted

# Шаг 2: Создаем класс для нейронной сети
class CaesarNet(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        emb = self.embeddings(x)
        out, hidden = self.rnn(emb)
        out = self.fc(out)
        return out

# Шаг 3: Генерируем обучающий датасет с зашифрованными и дешифрованными фразами
# shift = 3  # Задаем сдвиг
# phrases = ["hello", "world", "foo", "bar"]  # Оригинальные фразы
# encrypted_phrases = [caesar_cipher(x, shift) for x in phrases]  # Зашифрованные фразы

# # Создаем словарь символов и индексов
# CHARS = set(string.ascii_lowercase + ' ')
# CHAR_TO_INDEX = {char: i for i, char in enumerate(CHARS)}
# INDEX_TO_CHAR = {i: char for i, char in enumerate(CHARS)}
# vocab_size = len(CHARS)

# # Находим максимальную длину фразы
# max_sequence_length = max(len(seq) for seq in encrypted_phrases)

# Алфавит для шифрования (буквы и пробел)
CHARS = set(string.ascii_lowercase + ' ')
CHAR_TO_INDEX = {char: i for i, char in enumerate(CHARS)}
INDEX_TO_CHAR = {i: char for i, char in enumerate(CHARS)}

# Задаем сдвиг для шифра Цезаря
shift = 3

# Примеры входных и выходных данных
input_phrases = ["hello", "world", "foo", "bar"]
encrypted_phrases = [caesar_cipher(x, shift) for x in input_phrases]  # Зашифрованные фразы
decrypted_phrases = [caesar_cipher(x, -shift) for x in encrypted_phrases]  # Дешифрованные фразы

# Выводим на печать примеры шифрования и дешифрования
for i in range(len(input_phrases)):
    print(f"Input: '{input_phrases[i]}'")
    print(f"Encrypted: '{encrypted_phrases[i]}'")
    print(f"Decrypted: '{decrypted_phrases[i]}'")
    print()


# Создаем обучающий датасет
X = []
Y = []

for encrypted, original in zip(encrypted_phrases, phrases):
    input_seq = [CHAR_TO_INDEX[c] for c in encrypted]
    target_seq = [CHAR_TO_INDEX[c] for c in original]
    
    # Дополнение фразы пустыми символами до максимальной длины
    while len(input_seq) < max_sequence_length:
        input_seq.append(CHAR_TO_INDEX[' '])  # Добавляем пустой символ
    while len(target_seq) < max_sequence_length:
        target_seq.append(CHAR_TO_INDEX[' '])  # Добавляем пустой символ
    
    X.append(input_seq)
    Y.append(target_seq)

# Преобразуем в тензоры PyTorch
X = torch.tensor(X)
Y = torch.tensor(Y)

# Шаг 4: Обучаем нейронную сеть
embedding_dim = 32  # Размерность эмбеддингов
hidden_dim = 64  # Размерность скрытого состояния

model = CaesarNet(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Цикл обучения
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

# Шаг 5: Проверяем качество модели
test_data = ["hello", "world", "foo", "bar"]
encrypted_test = [caesar_cipher(x, shift) for x in test_data]

predicted = []
for x in encrypted_test:
    x = torch.tensor([CHAR_TO_INDEX[c] for c in x])
    pred = model(x)
    pred = pred.argmax(dim=-1)
    predicted.append(''.join([INDEX_TO_CHAR[ix.item()] for ix in pred]))

accuracy = sum(y == x for x, y in zip(phrases, predicted)) / len(test_data)
print(f"Accuracy: {accuracy * 100:.2f}%")


Input: 'hello'
Encrypted: 'khoor'
Decrypted: 'hello'

Input: 'world'
Encrypted: 'zruog'
Decrypted: 'world'

Input: 'foo'
Encrypted: 'irr'
Decrypted: 'foo'

Input: 'bar'
Encrypted: 'edu'
Decrypted: 'bar'

Accuracy: 100.00%


### Задание 2.
Выполнить практическую работу из лекционного ноутбука.

In [15]:
# Импорт необходимых библиотек
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim

# Загрузка данных из файла 'data.csv' с репликами из Симпсонов
df = pd.read_csv('data.csv')

# Извлечение предобработанных текстов и преобразование их в список
phrases = df['normalized_text'].dropna().tolist()

# Определение максимальной длины текста
MAX_LEN = 50

# Задание множества символов, которые будем использовать для кодировки
CHARS = 'abcdefghijklmnopqrstuvwxyz '

# Создание индекса для символов, включая 'none' как индекс для неизвестных символов
INDEX_TO_CHAR = ['none'] + list(CHARS)

# Создание словаря для символов и их индексов
CHAR_TO_INDEX = {char: i for i, char in enumerate(INDEX_TO_CHAR)}

# Предобработка данных
def preprocess_data(phrases):
    data = []
    for phrase in phrases:
        tokens = [CHAR_TO_INDEX.get(char, CHAR_TO_INDEX['none']) for char in phrase]
        if len(tokens) <= MAX_LEN:
            data.append(tokens)
    return data

# Преобразование текстов в численное представление
data = preprocess_data(phrases)

# Создание матрицы X, в которой строки представляют тексты, а столбцы - символы
X = torch.zeros((len(data), MAX_LEN), dtype=int)
for i, tokens in enumerate(data):
    X[i, :len(tokens)] = torch.tensor(tokens)

# Определение архитектуры модели RNN
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output)
        return output

# Создание модели, определение функции потерь и оптимизатора
model = RNNModel(len(INDEX_TO_CHAR), 128, len(INDEX_TO_CHAR))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Обучение модели
num_epochs = 20
batch_size = 100

for epoch in range(num_epochs):
    start_time = time.time()
    total_loss = 0
    total_batches = 0

    for batch_start in range(0, len(X), batch_size):
        batch_end = batch_start + batch_size
        if batch_end >= len(X):
            break
        batch = X[batch_start:batch_end]
        input_batch = batch[:, :-1]
        target_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        output = model(input_batch)
        output = output.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(output, target_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    elapsed_time = time.time() - start_time
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / total_batches:.3f}, Time: {elapsed_time:.3f} sec")

# Генерация текста
def generate_text(starting_text, max_length=100):
    model.eval()
    generated_text = list(starting_text)
    input_seq = torch.tensor([CHAR_TO_INDEX[char] for char in starting_text]).unsqueeze(0)

    for _ in range(max_length):
        output = model(input_seq)
        last_char_index = output[:, -1, :].argmax().item()
        last_char = INDEX_TO_CHAR[last_char_index]
        generated_text.append(last_char)
        input_seq = torch.tensor([last_char_index]).unsqueeze(0)

        if last_char == 'none':
            break

    return ''.join(generated_text)

# Генерация текста, начиная с 'dog'
generated_text = generate_text('dog')
print(generated_text)

# Генерация текста, начиная с 'It is'
generated_text = generate_text('It is')
print(generated_text)


Epoch 1/20, Loss: 1.585, Time: 25.410 sec
Epoch 2/20, Loss: 1.334, Time: 25.542 sec
Epoch 3/20, Loss: 1.260, Time: 24.145 sec
Epoch 4/20, Loss: 1.223, Time: 24.001 sec
Epoch 5/20, Loss: 1.200, Time: 24.711 sec
Epoch 6/20, Loss: 1.182, Time: 23.678 sec
Epoch 7/20, Loss: 1.168, Time: 23.631 sec
Epoch 8/20, Loss: 1.156, Time: 24.455 sec
Epoch 9/20, Loss: 1.146, Time: 24.134 sec
Epoch 10/20, Loss: 1.137, Time: 23.850 sec
Epoch 11/20, Loss: 1.129, Time: 24.166 sec
Epoch 12/20, Loss: 1.121, Time: 23.715 sec
Epoch 13/20, Loss: 1.114, Time: 23.565 sec
Epoch 14/20, Loss: 1.107, Time: 23.497 sec
Epoch 15/20, Loss: 1.101, Time: 23.544 sec
Epoch 16/20, Loss: 1.096, Time: 23.558 sec
Epoch 17/20, Loss: 1.090, Time: 23.772 sec
Epoch 18/20, Loss: 1.085, Time: 23.743 sec
Epoch 19/20, Loss: 1.080, Time: 23.774 sec
Epoch 20/20, Loss: 1.075, Time: 23.842 sec
dog whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe whe


KeyError: 'I'